In [2]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

In [3]:
path = '/home/esp-shared-a/Distribution/Workshops/cli_dy_summer_school_2023/observations/surface/era5'

In [ ]:
sst = xr.open_dataarray(f'{path}/era5_sst_1940_2022_1deg.nc',engine='netcdf4')

import cartopy.crs as ccrs
fig,ax = plt.subplots(2,figsize=(30,30),subplot_kw={'projection':ccrs.PlateCarree()})
im = ax[0].pcolormesh(sst.lon,sst.lat,sst.mean('time'))
im2 = ax[0].contour(sst.lon,sst.lat,sst.mean('time'),levels=(288,295))
plt.colorbar(im,ax=ax[0])
im = ax[1].pcolormesh(sst.lon,sst.lat,sst_dec.mean('time'),vmin=280,vmax=295)
im2 = ax[1].contour(sst.lon,sst.lat,sst_dec.mean('time'),levels=(285,290),c='r')
plt.colorbar(im,ax=ax[1])
gl = ax[1].gridlines(crs=ccrs.PlateCarree(), linewidth=2, color='black', alpha=0.5, linestyle='--', draw_labels=True)

for i in range(2):
    ax[i].set_extent((-80,-40,20,75))

In [ ]:
sst_dec = sst.where(sst.time.dt.month==12,drop=True)

In [ ]:
vgrad = (sst_dec.roll({'lat':1}) - sst_dec.roll({'lat':-1}))/2 

In [ ]:
fig,ax = plt.subplots(figsize=(10,10),subplot_kw={'projection':ccrs.PlateCarree()})
im = ax.pcolormesh(sst.lon,sst.lat,vgrad.mean('time'))
im2 = ax.contour(sst.lon,sst.lat,sst_dec.mean('time'),levels=(1,288),c='r')

ax.set_extent((-80,-40,20,75))
plt.colorbar(im,ax=ax)

In [ ]:
# define mean vertical gradient, and find maximum within box (-75,-50,20,50)

vgrad_mean = vgrad.mean('time').sel(lat = slice(20,50), lon = slice(360-75,360-50))


In [ ]:
lats_max = vgrad_mean.argmax(dim='lat')

In [ ]:
sst.lat

In [ ]:
fig,ax = plt.subplots(figsize=(10,10),subplot_kw={'projection':ccrs.PlateCarree()})
im = ax.pcolormesh(vgrad_mean.lon,vgrad_mean.lat,vgrad_mean)
im2 = ax.plot(vgrad_mean.lon,vgrad_mean.lat.isel(lat = lats_max),transform=ccrs.PlateCarree())
im2 = ax.contour(sst.lon,sst.lat,sst_dec.mean('time'),levels=20,c='r')
ax.set_extent((-80,-40,20,50))
ax.coastlines()
plt.colorbar(im,ax=ax)

In [ ]:
def get_gs_coords(da, m):
    
    '''
    INPUTS:
    da: a dataarray containing sst on a regular grid (dimensions nx * ny)
    m: desired month for calculation
    
    OUTPUTS:
    
    gs_coords: a dataarray containing lat/lon of maximum sst meridional gradient (dimensions nx * 2)
    
    '''
    
    #da_mean = da.where(da.time.dt.month == m,drop=True).mean('time')
    da_mean = da[m]
    
    da_grad = (da_mean.roll({'lat':1}) - da_mean.roll({'lat':-1}))/2 
    
    da_grad = da_grad.sel(lat = slice(20,50), lon = slice(360-75,360-50))
    
    lats_max = da_grad.argmax(dim='lat')
    
    lats_max = da_grad.lat.isel(lat = lats_max)
    
    return lats_max
    
    

In [ ]:
test = get_gs_coords(sst,12)

In [ ]:
test

In [ ]:
fig,axs = plt.subplots(4,3,figsize=(20,20),subplot_kw={'projection':ccrs.PlateCarree()})
from matplotlib import colors
norm = colors.Normalize(2,5)

for i,ax in enumerate(axs.flatten()):
    sst_grad = (sst.roll({'lat':1}) - sst.roll({'lat':-1}))/2 
    im = ax.pcolormesh(sst.lon,sst.lat,sst_grad.where(sst.time.dt.month == i+1,drop=True).mean('time'),norm=norm)
    lats_max = get_gs_coords(sst,i+1)
    ax.plot(lats_max.lon,lats_max.lat,'.r',transform=ccrs.PlateCarree())

    ax.coastlines()
    ax.set_extent((-80,-40,20,50))
plt.colorbar(im,ax=ax)

In [ ]:
import haversine as hs   
from haversine import Unit

In [ ]:
def get_total_gradient(da, n_roll):
    
    
    
    #get haversine distance between longitudes
    
    lats = da.lat
    lats0 = np.squeeze(np.stack((lats,np.zeros(len(lats)))))
    lats1 = np.squeeze(np.stack((lats,np.ones(len(lats)))))
    xdist = hs.haversine(lats0,lats1,Unit.KILOMETERS)
    ydist = 111*(lats[1]-lats[0]) # take distance between latitudes as 111 km
    
    ygrad = (da_mean.roll({'lat':n_roll}) - da_mean.roll({'lat':-n_roll}))/(2 * n_roll * ydist)
    xgrad = (da_mean.roll({'lon':n_roll}) - da_mean.roll({'lon':-n_roll}))/(2 * n_roll * xdist[None,:,None])

    grad_tot = (xgrad**2 + ygrad**2)**0.5
    
    return grad_tot

In [ ]:
x,y = np.meshgrid(sst.lon,sst.lat)

print(hs.haversine((35,0),(35,1)))
print(hs.haversine((45,0),(45,1)))